In [23]:
from dotenv import load_dotenv
import os
import openai
import json
import gradio as gr
import logging
from datetime import datetime

# Load environment variables from .env file
load_dotenv()

True

In [24]:
openai.api_type = os.getenv("a_type")
openai.api_base = os.getenv("base")
openai.api_version = os.getenv("version")
openai.api_key = os.getenv("key")


In [25]:

# Path to the knowledge base file
KNOWLEDGE_BASE_PATH = "faq_json.json"

# Configure logging
logging.basicConfig(
    filename='app.log',  # Log file name
    level=logging.INFO,   # Log level
    format='%(asctime)s - %(levelname)s - %(message)s'  # Log format
)

# Load knowledge base (with error handling)
def load_knowledge_base():
    if os.path.exists(KNOWLEDGE_BASE_PATH):
        try:
            with open(KNOWLEDGE_BASE_PATH, "r") as f:
                return json.load(f)
        except json.JSONDecodeError:
            logging.error(f"{KNOWLEDGE_BASE_PATH} is not a valid JSON file. Initializing an empty knowledge base.")
            return {"FAQs": []}
    else:
        logging.warning(f"{KNOWLEDGE_BASE_PATH} not found. Initializing an empty knowledge base.")
        return {"FAQs": []}

knowledge_base = load_knowledge_base()

# Function to log interactions using the logging module
def log_interaction(query, response):
    logging.info(f"Query: {query}, Response: {response}")

# Function to search the knowledge base for an exact match
def search_knowledge_base(query):
    for faq in knowledge_base.get("FAQs", []):
        if faq["question"].strip().lower() == query.strip().lower():
            return faq["answer"]
    return None

# Function to provide context from the knowledge base to the LLM
def get_knowledge_base_context():
    context = "Here are some frequently asked questions and their answers:\n"
    for faq in knowledge_base.get("FAQs", []):
        context += f"Q: {faq['question']}\nA: {faq['answer']}\n\n"
    return context

# Function to handle user queries
def handle_query(query):
    if not query:
        return "Please provide your questions."

    # Step 1: Search the knowledge base for an exact match
    kb_answer = search_knowledge_base(query)
    
    if kb_answer:
        log_interaction(query, kb_answer)
        return kb_answer

    # Step 2: If no exact match, use the LLM with knowledge base context
    try:
        context = get_knowledge_base_context()
        persona=f'''
        You are a Customer Service Assistant. Your primary role is to assist users by answering questions based on the provided FAQs.
        Below are the following some FAQs: 
        {context}
--------------------
Guidelines:

Answer Only from FAQs: Use the given FAQs to respond to customer inquiries. Do not provide information outside the provided context.
Firm but Polite Boundaries: If a user asks a question unrelated to customer service FAQs, respond with:
"I am customer service chatbot.I'm here to assist with customer service-related queries only."
Ensure Consistency: Always provide accurate and concise answers directly from the FAQs without unnecessary elaboration.
No Speculation: If a relevant FAQ does not exist for a query, do not attempt to infer or create an answer. Instead, guide the user to contact customer support if needed.'''
        response = openai.ChatCompletion.create(
            engine="GPT4-O",
            messages=[
                {"role": "system", "content":persona},
                {"role": "user", "content": query}
                
            ],
            temperature=0.7,
            max_tokens=4096,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
        
        llm_answer = response.choices[0].message.content.strip()
        
    except Exception as e:
        logging.error(f"Error querying LLM: {e}")
        llm_answer = "I'm sorry, I couldn't find a clear answer to your question."

    # Log the interaction with LLM's answer
    log_interaction(query, llm_answer)

    # Step 3: Add Q&A to knowledge base if it's not present and LLM provided an answer
    if llm_answer and not kb_answer:  # Only add if there's no existing answer
        new_faq = {"question": query, "answer": llm_answer}
        knowledge_base["FAQs"].append(new_faq)
        
        # Save updated knowledge base back to JSON file
        with open(KNOWLEDGE_BASE_PATH, "w") as f:
            json.dump(knowledge_base, f, indent=4)

    return llm_answer

# Gradio interface setup
def gradio_interface(query):
    return handle_query(query)

# Create Gradio UI components
iface = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Textbox(label="Your Question"),
    outputs="text",
    title="Intelligent FAQ Assistant",
    description="Ask questions related to our FAQs."
)

# Launch the Gradio app
if __name__ == "__main__":
    iface.launch()


Running on local URL:  http://127.0.0.1:7875

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 4.5.0, however version 4.44.1 is available, please upgrade.
--------
